In [65]:
import json
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics


In [11]:
data_dir =  'C:\\UTD\\Sophomore Year\\Projects\\exoplanet-atmosphere-analysis-with-ML\\' 
df = pd.read_csv ('ExoplanetDataset.csv', encoding = 'utf-8-sig')
df.columns

Index(['rowid', 'pl_name', 'hostname', 'pl_letter', 'hd_name', 'hip_name',
       'tic_id', 'gaia_id', 'sy_snum', 'sy_pnum',
       ...
       'sy_kepmag', 'sy_kepmagerr1', 'sy_kepmagerr2', 'sy_kepmag_reflink',
       'pl_nnotes', 'st_nphot', 'st_nrvc', 'st_nspec', 'pl_nespec',
       'pl_ntranspec'],
      dtype='object', length=313)

In [50]:
#Removing columns that are really unnecessary
clean_df = df[df.columns.drop(list(df.filter(regex='_ref')))]
clean_df = clean_df[clean_df['pl_eqt'].notna()]
clean_df = clean_df[clean_df['pl_dens'].notna()]
clean_df = clean_df[clean_df['pl_bmasse'].notna()]
clean_df = clean_df[clean_df['st_lum'].notna()]

In [51]:
#Extracting features to run a dirty regression model on temperature
#Columns/Features used: planet density, core mass, stellar luminosity
#Testing feature: Equilibrium Temperature
X = clean_df[['pl_dens', 'pl_bmasse','st_lum']]
y = clean_df[['pl_eqt']]

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [53]:
X['st_lum'].isnull().values.any()

False

In [59]:
#Attempt One - simple random forest regression using all features
tempDetect = RandomForestRegressor(n_estimators = 100, random_state = 0)
tempDetect.fit(X_train, y_train)

<ipython-input-59-b2624ec70fae>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  tempDetect.fit(X_train, y_train)


RandomForestRegressor(random_state=0)

In [66]:
y_pred = tempDetect.predict(X_test)
# View accuracy score
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
mape = np.mean(np.abs((y_test - y_pred) / np.abs(y_test)))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))
print('Accuracy:', round(100*(1 - mape), 2))

Mean Absolute Error (MAE): 252.5976176819855
Mean Squared Error (MSE): 112524.58141869027
Root Mean Squared Error (RMSE): 335.44683843895484


ValueError: Unable to coerce to Series, length must be 1: given 1203